# Semantic Segmentation

Semantic segmentation, or image segmentation, is the task of clustering parts of an image together which belong to the same object class. In this tutorial, we will...

**[Part 1](#Part-1)

1. Learn about the [DynamicEarthNet Challenge](#1.-DynamicEarthNet-Challenge),
2. Meet the [data](#2.-DynamicEarthNet:-Daily-Multi-Spectral-Satellite-Dataset-for-Semantic-Change-Segmentation),
3. Explore a [random forest-based segmentation model](#3.-Random-Forest)
4. Design a simple artificial neural network-based solution.

**[Part 2](#Part-2)

5. 

## 1. DynamicEarthNet Challenge

Remote sensing is entering a new era of time-series analysis. Short revisit times of satellites allow for monitoring of many areas across the globe on a weekly basis. However, there has been little exploration of deep learning techniques to leverage this new temporal dimension at scale. Especially, existing approaches have struggled to combine the power of different sensors to make use of all available information. In addition, large scale high quality change detection benchmarks are rare. To stimulate innovation in spatio-temporal machine learning, we have partnered up to propose a unique challenge centered around modeling multi-temporal land cover changes from Planetscope and Sentinel time series data.

Do you want to know more? Then, visit the [DynamicEarthNet](https://codalab.lisn.upsaclay.fr/competitions/2882) web page!

![Data sample](challenge_sample_cube.gif)

## 2. DynamicEarthNet: Daily Multi-Spectral Satellite Dataset for Semantic Change Segmentation

**#TODO** Add a description

More details can be found in the original [data set paper](https://openaccess.thecvf.com/content/CVPR2022/papers/Toker_DynamicEarthNet_Daily_Multi-Spectral_Satellite_Dataset_for_Semantic_Change_Segmentation_CVPR_2022_paper.pdf).

```
@inproceedings{toker2022dynamicearthnet,
  title={DynamicEarthNet: Daily Multi-Spectral Satellite Dataset for Semantic Change Segmentation},
  author={Toker, Aysim and Kondmann, Lukas and Weber, Mark and Eisenberger, Marvin and Camero, Andr{\'e}s and Hu, Jingliang and Hoderlein, Ariadna Pregel and {\c{S}}enaras, {\c{C}}a{\u{g}}lar and Davis, Timothy and Cremers, Daniel and others},
  booktitle={Proceedings of the IEEE/CVF Conference on Computer Vision and Pattern Recognition},
  pages={21158--21167},
  year={2022}
}

```

In [ ]:
# install the required packages using pip install -r requirements.txt
# import sys
# !{sys.executable} -m pip install -r requirements.txt

In [ ]:
import h5py
import numpy as np

import matplotlib.pyplot as plt
from matplotlib import cm

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from functools import reduce

np.random.seed(3108)

In [ ]:
# this should match the name of the data sets
filename = 'data/dynamic_earth_net.h5'

dataset = h5py.File(filename, 'r')

# show the content names
print(list(dataset.keys()))

In [ ]:
# load Sentinel-2 data
# O'Brien Mountain, California, US. Check it on [Google Maps](https://goo.gl/maps/FwK3my9AX1xZ2Jrs5)
sen2_train = np.array(dataset['s2_train'])

print("Sentinel-2 shape: ", sen2_train.shape)
# 24 samples, 1024x1024 pixels, 12 bands

def true_color(X):
    blue = ((X[:,:,0] - np.min(X[:,:,0])) / (np.max(X[:,:,0]) - np.min(X[:,:,0])))
    green = ((X[:,:,1] - np.min(X[:,:,1])) / (np.max(X[:,:,1]) - np.min(X[:,:,1])))
    red = ((X[:,:,2] - np.min(X[:,:,2])) / (np.max(X[:,:,2]) - np.min(X[:,:,2])))
    return np.dstack((red, green, blue))  

index = 2

fig=plt.figure(figsize=(12, 12))
# B2 = blue, B3 = green, and B4 = red. Note that B1 was not included in the data set
plt.imshow(true_color(sen2_train[index]))
plt.title('Sentinel-2')
plt.show()

In [ ]:
# load Planet Fusion data
planet_train = np.array(dataset['planet_train'])

print("Planet Fusion shape: ", planet_train.shape)
# 24 samples, 1024x1024 pixels, 4 bands

index = 2

fig=plt.figure(figsize=(12, 12))
# B1 = blue, B2 = green, B3 = red, and B4= NIR
plt.imshow(true_color(planet_train[index]))
plt.title('Planet Fusion')
plt.show()

In [ ]:
# Ocala area, Florida, US. You can check it on [Google Maps](https://goo.gl/maps/uXmmcdaniRDiQEqA6)
sen2_test = np.array(dataset['s2_test'])
planet_test = np.array(dataset['planet_test'])

fig=plt.figure(figsize=(12, 6))
fig.add_subplot(1, 2, 1)
plt.imshow(true_color(sen2_test[index]))
plt.title('Sentinel-2')
fig.add_subplot(1, 2, 2)
plt.imshow(true_color(planet_test[index]))
plt.title('Planet Fusion')
plt.show()

In [ ]:
# load Planet Fusion data
labels_train = np.array(dataset['labels_train'])
labels_test = np.array(dataset['labels_test'])

print("Labels shape: ", labels_train.shape)
# 24 samples, 1024x1024 pixels, 12 bands

color_map = cm.get_cmap("Accent", lut=labels_train.shape[-1])
labels=['impervious surface',
       'agriculture',
       'forest & other vegetation',
       'wetlands',
       'soil',
       'water',
       'snow & ice']

def labels2rgb(
    X,
    colors=color_map.colors):
    Z = np.zeros((X.shape[0], X.shape[1], 3))
    for i in range(X.shape[0]):
        for j in range(X.shape[1]):
            Z[i,j] = colors[np.argmax(X[i,j]), :3]
    return Z

fig=plt.figure(figsize=(12, 12))
plt.imshow(labels2rgb(labels_train[index]), cmap=color_map)
plt.title('Labels')
cbar = plt.colorbar(ticks=np.linspace(1/14, 13/14, 7))
cbar.set_ticklabels(labels)
plt.show()

In [ ]:
fig=plt.figure(figsize=(12, 12))
plt.imshow(labels2rgb(labels_test[index]), cmap=color_map)
plt.title('Labels')
cbar = plt.colorbar(ticks=np.linspace(1/14, 13/14, 7))
cbar.set_ticklabels(labels)
plt.show()

One key application of our dataset is to measure how a given local region changes over time. For the standard task of binary change detection, we classify each pixel into change or no-change. This definition, however, disregards semantic information. We, therefore, generalize this classical notion to a multi-class segmentation task, which we refer to as semantic change segmentation.

In [ ]:
fig=plt.figure(figsize=(12, 12))

for i in range(2,6):
    fig.add_subplot(2, 2, i-1)
    plt.imshow(labels2rgb(labels_train[i]), cmap=color_map)

plt.show()

## 3. Random Forest Segmenter

Let's start with a very simple approach, namely using a *Random Forest Classifier* to segment an image. Loosely speaking, we will "classify" each pixel into one of the semantic segmentation categories.

In [ ]:
def labels2int(X):
    Z = np.zeros(X.shape[:3])    
    for i in range(X.shape[0]):
        for j in range(X.shape[1]):
            for k in range(X.shape[2]):
                Z[i,j,k] = np.argmax(X[i,j,k,:]) + 1
    return Z

labels_train_int = labels2int(labels_train)

We now need to format the data so that it fits to what the random forest classifier expects, namely an array of shape (n, m) as input data, and (n) annotations. Note that n corresponds to number of pixels and m to number of features (in our case, bands or channels).

In [ ]:
y = np.reshape(labels_train_int[0], 
                                   (reduce(lambda x, y: x*y, labels_train_int[0].shape)))
print("Labels flattened shape", y.shape)

X = np.reshape(planet_train[0], (reduce(lambda x, y: x*y, planet_train[0].shape[:-1]),
                                              reduce(lambda x, y: x*y, planet_train[0].shape[-1:])))
print("Train flattened shape", X.shape)

Let's train a *Random Forest Classifier* to segment an image. Note that we are training on a single image.

In [ ]:
rfc = RandomForestClassifier(random_state=3108)
rfc.fit(X, y)

Now, we will use the trained model to segment a new image.

In [ ]:
Z = np.reshape(planet_train[-1], (reduce(lambda x, y: x*y, planet_train[-1].shape[:-1]),
                                              reduce(lambda x, y: x*y, planet_train[-1].shape[-1:])))
yz = rfc.predict(Z)
yz_img = np.reshape(yz, (labels_train_int[-1].shape[0], labels_train_int[-1].shape[1]))

In [ ]:
fig = plt.figure(figsize=(12, 12))

fig.add_subplot(1, 2, 1)
plt.imshow(labels_train_int[-1], cmap=color_map)

fig.add_subplot(1, 2, 2)
plt.imshow(yz_img, cmap=color_map)

plt.show()


On the left side, the ground truth is show, while on the right side the predicted segmentation is shown. Qualitatively, the results are quite OK. So, how about a quantitative appraisal?

There are different metrics to evaluate the performance of a semantic segmentation:
1. Pixel accuracy: Is the percent of pixels in your image that are classified correctly.
2. Intersection-Over-Union (IoU, Jaccard Index): The IoU is the area of overlap between the predicted segmentation and the ground truth divided by the area of union between the predicted segmentation and the ground truth.
3. Dice coefficient (F1 Score): Is two times the area of overlap divided by the total number of pixels in both images.

In [ ]:
print("Accuracy: %5.4f" % metrics.accuracy_score(labels_train_int[1].flatten(), yz))
print("IoU: %5.4f" % metrics.jaccard_score(labels_train_int[1].flatten(), yz, average="weighted"))
print("F1 Score: %5.4f" % metrics.f1_score(labels_train_int[1].flatten(), yz, average="weighted"))

Performance is overall quite good (in-line with the qualitative evaluation). However, it is relatively easy to observe that two classes are *predominant* (aka. majority class). What is happening to individual classes?

In [ ]:
print(metrics.f1_score(labels_train_int[-1].flatten(), yz, average=None))

## Do it yourself

Train a *Random Forest Classifier* to segment using the test image(s), and compare the performance with respect to the example shown above.

## 4. Simple CNN-based approach

We will implement a FCN (fully convolutional network)

In [ ]:
def FCN_model(len_classes=7, dropout_rate=0.2):        
    input = tf.keras.layers.Input(shape=(1024, 1024, 4))

    # A convolution block
    x = tf.keras.layers.Conv2D(filters=32, kernel_size=3, strides=1, padding="same")(input)
    x = tf.keras.layers.Dropout(dropout_rate)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)
    
    # Fully connected layer 1
    x = tf.keras.layers.Conv2D(filters=64, kernel_size=1, strides=1)(x)
    x = tf.keras.layers.Dropout(dropout_rate)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)
    
    # Fully connected layer 2
    x = tf.keras.layers.Conv2D(filters=len_classes, kernel_size=1, strides=1)(x)
    x = tf.keras.layers.Dropout(dropout_rate)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    # x = tf.keras.layers.GlobalMaxPooling2D()(x)
    
    predictions = tf.keras.layers.Activation('softmax')(x)
    
    model = tf.keras.Model(inputs=input, outputs=predictions)
    
    return model   


In [ ]:
fcn = FCN_model()
print(fcn.summary())

In [ ]:
fcn.compile(optimizer='adam',
            loss='categorical_crossentropy',
            metrics=['accuracy'])

In [ ]:
X = planet_train[0:3]
y = labels_train[0:3]

In [ ]:
fcn.fit(X, y, epochs=5)

In [ ]:
planet_train[-1:].shape

In [ ]:
yz_fcn = fcn.predict(planet_train[-1:])
# yz_fcn = np.squeeze(yz_fcn)

In [ ]:
fig = plt.figure(figsize=(12, 12))

fig.add_subplot(1, 2, 1)
plt.imshow(labels_train_int[-1], cmap=color_map)

fig.add_subplot(1, 2, 2)
plt.imshow(labels2int(yz_fcn)[0], cmap=color_map)

plt.show()

In [59]:
yz_fcn_int = labels2int(yz_fcn)[0]

print("Accuracy: %5.4f" % metrics.accuracy_score(labels_train_int[1].flatten(), yz_fcn_int.flatten()))
print("IoU: %5.4f" % metrics.jaccard_score(labels_train_int[1].flatten(), yz_fcn_int.flatten(), average="weighted"))
print("F1 Score: %5.4f" % metrics.f1_score(labels_train_int[1].flatten(), yz_fcn_int.flatten(), average="weighted"))

Accuracy: 0.2488
IoU: 0.1527
F1 Score: 0.2461
